In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
import ee
import geemap
from utils import ee_utils, date_utils

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

In [4]:
ee.Initialize(project="thurgau-irrigation")

## Validating ET Blue Maps with Walter Koch's Estimates

This notebook follows a three-step process:

1. **Create Yearly ET Blue Maps (2018-2022):**  
   - Aggregate dekadal ET Blue maps into monthly maps.
   - Filter the maps to include only the growing season months (April through October).
   - Sum the monthly ET Blue maps to produce yearly ET Blue maps.

2. **Calculate Yearly ET Blue Values for Potentially Irrigated Fields:**  
   - Compute ET blue in m3/ha/yr for each crop field for each year.

3. **Compare ET Blue Values with Walter Koch's Estimates:**
   - Make pretty plots

## Aggregate dekadal ET Blue maps into monthly maps

In [5]:
ET_blue_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_blue_2018-2022")

def back_to_float(image: ee.Image): 
    date = image.get("system:time_start")
    return image.toFloat().divide(100).set("system:time_start", date)

ET_blue_collection = ET_blue_collection.map(back_to_float)

In [6]:
date_utils.print_collection_dates(ET_blue_collection)

Dates of images in the collection:
2018-01-01
2018-01-11
2018-01-21
2018-02-01
2018-02-11
2018-02-21
2018-03-01
2018-03-11
2018-03-21
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2018-10-01
2018-10-11
2018-10-21
2018-11-01
2018-11-11
2018-11-21
2018-12-01
2018-12-11
2018-12-21
2019-01-01
2019-01-11
2019-01-21
2019-02-01
2019-02-11
2019-02-21
2019-03-01
2019-03-11
2019-03-21
2019-04-01
2019-04-11
2019-04-21
2019-05-01
2019-05-11
2019-05-21
2019-06-01
2019-06-11
2019-06-21
2019-07-01
2019-07-11
2019-07-21
2019-08-01
2019-08-11
2019-08-21
2019-09-01
2019-09-11
2019-09-21
2019-10-01
2019-10-11
2019-10-21
2019-11-01
2019-11-11
2019-11-21
2019-12-01
2019-12-11
2019-12-21
2020-01-01
2020-01-11
2020-01-21
2020-02-01
2020-02-11
2020-02-21
2020-03-01
2020-03-11
2020-03-21
2020-04-01
2020-04-11
2020-04-21
2020-05-01
2020-05-11
2020-05-21
2020-06-

In [7]:
ET_blue_collection_monthly = ee_utils.aggregate_to_monthly(
    ET_blue_collection, bands=["ET_blue", "ET_blue_m3"]
)

## Filter the maps to include only the growing season months (April through October).

In [8]:
ET_blue_collection_monthly_growing_season = ET_blue_collection_monthly.filter(ee.Filter.calendarRange(4, 10, "month"))

# date_utils.print_collection_dates(ET_blue_collection_monthly_growing_season)

Dates of images in the collection:
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2021-04-01
2021-05-01
2021-06-01
2021-07-01
2021-08-01
2021-09-01
2021-10-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01


## Sum the monthly ET Blue maps to produce yearly ET Blue maps.

In [9]:
def create_growing_season_yearly_sums(
    collection: ee.ImageCollection,
) -> ee.ImageCollection:
    """
    Create yearly sums from a collection of monthly images for the growing season (April to October).

    Args:
        collection (ee.ImageCollection): Input collection with monthly images for the growing season.

    Returns:
        ee.ImageCollection: Collection with yearly sum images for the growing season.
    """

    def sum_growing_season(year):
        start_date = ee.Date.fromYMD(year, 4, 1)  # April 1st
        end_date = ee.Date.fromYMD(year, 11, 1)  # November 1st (exclusive)
        yearly_sum = collection.filterDate(start_date, end_date).sum()
        return yearly_sum.set("year", year).set(
            "system:time_start", start_date.millis()
        )

    # Get the range of years in the collection
    start_year = ee.Date(collection.first().get("system:time_start")).get("year")
    end_year = ee.Date(
        collection.sort("system:time_start", False).first().get("system:time_start")
    ).get("year")

    # Create a list of years
    years = ee.List.sequence(start_year, end_year)

    # Map over the years to create yearly sums
    yearly_sums = ee.ImageCollection.fromImages(years.map(sum_growing_season))

    return yearly_sums


ET_blue_collection_yearly = create_growing_season_yearly_sums(
    ET_blue_collection_monthly_growing_season
)

In [10]:
# ET_blue_collection_yearly.first().bandNames().getInfo()

## Compute ET blue in m3/ha/yr for each crop field for each year and export

In [11]:
def calculate_et_blue_per_field(
    et_blue_image: ee.Image, crop_fields: ee.FeatureCollection
) -> ee.FeatureCollection:
    """
    Calculate ET_blue in m3/ha/yr for each crop field and add it as a property.

    Args:
        et_blue_image (ee.Image): Image containing ET_blue estimates in m3/yr.
        crop_fields (ee.FeatureCollection): Collection of crop field features.

    Returns:
        ee.FeatureCollection: Updated crop fields with ET_blue_m3_ha_yr property.
    """

    def calculate_et_blue(feature: ee.Feature) -> ee.Feature:
        # Get the area of the feature in hectares
        area_ha = ee.Number(feature.get("Area")).divide(10000)  # Convert m² to ha

        # Calculate the sum of ET_blue within the feature
        et_blue_sum = et_blue_image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            scale=10,  # Use the resolution of the ET_blue image
            maxPixels=1e9,
        ).get("ET_blue_m3")

        et_blue_sum = ee.Number(ee.Algorithms.If(et_blue_sum, et_blue_sum, 0))

        # Calculate ET_blue in m3/ha/yr
        et_blue_m3_ha_yr = ee.Number(et_blue_sum).divide(area_ha).toInt()

        # Add the result as a new property
        return feature.set("ET_blue_m3_ha_yr", et_blue_m3_ha_yr)

    # Map the calculation over all features
    return crop_fields.map(calculate_et_blue)

In [12]:
def export_feature_collection(collection: ee.FeatureCollection, task_name: str, asset_id: str): 
    """
    Export the feature collection to an Earth Engine asset.

    Args:
        collection: The feature collection to export.
        year: The year of the feature collection.
        task_name: The name of the export task.
        asset_id: The asset ID to export to.
    """
    task = ee.batch.Export.table.toAsset(
        collection=collection,
        description=task_name,
        assetId=asset_id,
    )
    task.start()

In [13]:
years = range(2020, 2023)

not_irrigated_crops = get_crops_to_exclude()
exclude_filter, _ = create_crop_filters(not_irrigated_crops, not_irrigated_crops)

ET_blue_collection_yearly_list = ET_blue_collection_yearly.toList(
    ET_blue_collection_yearly.size()
)

for year in years:
    index = 2018 - year
    nutzung_collection_year = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
    )

    potentially_rainfed, _ = filter_crops(nutzung_collection_year, exclude_filter, _)

    et_blue_image = ee.Image(ET_blue_collection_yearly_list.get(index))

    crop_with_et_blue = calculate_et_blue_per_field(et_blue_image, potentially_rainfed)

    task_name = f"ET_blue_m3_ha_yr_per_field_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_yr_per_field_2018-2022/{task_name}"

    export_feature_collection(crop_with_et_blue, task_name, asset_id)
    print(f"Exporting {task_name} to {asset_id}")

AttributeError: 

In [ ]:
# File: /src/visualization/et_blue_comparison.py

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Generate dummy data
np.random.seed(42)

# Calculated ET_blue estimates for each crop type (m3/ha/year)
crop_types = ['A', 'B', 'C']
calculated_et_blue = {
    'A': np.random.uniform(100, 500, 100),
    'B': np.random.uniform(200, 600, 100),
    'C': np.random.uniform(300, 700, 100)
}

# Farmer's estimates (ranges) for each crop type (m3/ha/year)
farmer_estimates = {
    'A': (150, 450),
    'B': (250, 550),
    'C': (350, 650)
}

def plot_boxplot_comparison(calculated_et_blue, farmer_estimates):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot calculated ET_blue boxplots
    bp = ax.boxplot([calculated_et_blue[crop] for crop in crop_types], positions=range(1, 4), widths=0.6)
    
    # Plot farmer's estimate ranges
    for i, crop in enumerate(crop_types):
        ax.plot([i+0.7, i+1.3], [farmer_estimates[crop][0], farmer_estimates[crop][0]], 'r--')
        ax.plot([i+0.7, i+1.3], [farmer_estimates[crop][1], farmer_estimates[crop][1]], 'r--')
    
    ax.set_xlabel('Crop Type')
    ax.set_ylabel('ET Blue (m³/ha/year)')
    ax.set_title('Comparison of Calculated ET Blue vs Farmer Estimates')
    ax.set_xticks(range(1, 4))
    ax.set_xticklabels(crop_types)
    ax.legend([bp["boxes"][0], plt.Line2D([0], [0], color='r', linestyle='--')], 
              ['Calculated ET Blue', 'Farmer Estimate Range'])
    
    plt.tight_layout()
    plt.show()

def plot_violin_comparison(calculated_et_blue, farmer_estimates):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot calculated ET_blue violin plots
    parts = ax.violinplot([calculated_et_blue[crop] for crop in crop_types], positions=range(1, 4), showmeans=True, showmedians=True)
    
    # Plot farmer's estimate ranges
    for i, crop in enumerate(crop_types):
        ax.plot([i+0.7, i+1.3], [farmer_estimates[crop][0], farmer_estimates[crop][0]], 'r--')
        ax.plot([i+0.7, i+1.3], [farmer_estimates[crop][1], farmer_estimates[crop][1]], 'r--')
    
    ax.set_xlabel('Crop Type')
    ax.set_ylabel('ET Blue (m³/ha/year)')
    ax.set_title('Comparison of Calculated ET Blue vs Farmer Estimates')
    ax.set_xticks(range(1, 4))
    ax.set_xticklabels(crop_types)
    ax.legend([parts["bodies"][0], plt.Line2D([0], [0], color='r', linestyle='--')], 
              ['Calculated ET Blue', 'Farmer Estimate Range'])
    
    plt.tight_layout()
    plt.show()

def plot_histogram_comparison(calculated_et_blue, farmer_estimates):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Comparison of Calculated ET Blue vs Farmer Estimates')
    
    for i, crop in enumerate(crop_types):
        sns.histplot(calculated_et_blue[crop], kde=True, ax=axs[i])
        axs[i].axvline(farmer_estimates[crop][0], color='r', linestyle='--')
        axs[i].axvline(farmer_estimates[crop][1], color='r', linestyle='--')
        axs[i].set_xlabel('ET Blue (m³/ha/year)')
        axs[i].set_ylabel('Frequency')
        axs[i].set_title(f'Crop Type {crop}')
    
    plt.tight_layout()
    plt.show()

# Generate the plots
plot_boxplot_comparison(calculated_et_blue, farmer_estimates)
plot_violin_comparison(calculated_et_blue, farmer_estimates)
plot_histogram_comparison(calculated_et_blue, farmer_estimates)